# Hands-on

#### Importando bibliotecas

In [ ]:
import pandas as pd
import numpy as np
# demais pacotes serão importados, mas vamos adicioná-los à medida que houver necessidade 

#### Adquirindo os dados

Vamos fazer a leitura de nossos dados utilizando Pandas

In [ ]:
data = pd.read_csv('bases/forestfires.csv',sep=';')

# Imprimindo as primeiras 5 linhas
data.head()

In [ ]:
# podemos usar o método info() que é útil para obter uma rápida descrição dos dados, em particular o número de linhas, o tipo de cada 
# atributo e o número de valores não nulos
data.info()

In [ ]:
data['month'].value_counts()

In [ ]:
# O método describe() apresenta um resumo dos atributos numéricos
data.describe()

In [ ]:
# Função de apoio
# Não se preocupe se não entender como funciona o plot de gráficos agora. Vamos estudar isso em detalhes na próxima aula
# deixei aqui apenas para detalhar um pouco mais nosso trabalho

import matplotlib.pyplot as plt
data.hist(bins=50, figsize=(20,15))
plt.show()

### Correlação

Podemos calcular o coeficiente de correlação padrão (Correlação de Pearson)entre cada par de atributos usando o método corr(). Vamos ver o quanto cada atributo se relaciona com area (aquilo que estamos tentando predizer)

In [ ]:
corr_matrix = data.corr()
corr_matrix["area"].sort_values(ascending=False)

## Preparando os Dados para Algoritmos de Machine Learning

In [ ]:
features = data.drop('area', axis=1)
labels = data['area'].copy()

### Limpeza dos Dados

In [ ]:
data.info()

In [ ]:
# data.dropna(subset=["temp"]) # opção 1
# data.drop("temp", axis=1) # opção 2
# median = data["temp"].median()
# data["temp"].fillna(median) # opção 3

# lembrando que se a opção 3 for escolhida, precisamos tratar os dados tanto no conjunto de treino quanto no conjunto de teste

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='median')

In [ ]:
data_num = data.drop(['month','day'],axis=1)

In [ ]:
# Agora precisamos ajustar a instância do Imputer aos dados de treinamento usando o método fit():
imputer.fit(data_num)

In [ ]:
imputer.statistics_

In [ ]:
X = imputer.transform(data_num) # o resultado é um numpy.array. Precisamos converter para DataFrame
data_transformed = pd.DataFrame(X, columns=data_num.columns)
data_transformed.head()

In [ ]:
# podemos confirmar que os valores faltantes foram todos preenchidos
data_transformed.info()

### Manipulando Atributos Textuais e Categóricos

In [ ]:
# separando apenas atributos categóricos
data_cat = data[['month','day']]

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
data_cat_1hot = encoder.fit_transform(data_cat)
data_cat_1hot

### Pipelines de Transformação

In [ ]:
from sklearn.pipeline import Pipeline
num_pipeline = Pipeline([
    ('imputer',SimpleImputer(missing_values=np.nan, strategy='median')),
])
data_num_tr = num_pipeline.fit_transform(data_num)

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

A classe DataFrameSelector irá separar o dataframe de acordo com o conteúdo da variável attribute_names, retornando apenas aquelas que são categórias ou aquelas que são numéricas. Essa classe herda os métodos BaseEstimator e TransformerMixin.

O primeiro é responsável por implementar os métodos get_params() e set_params() que são extremamente úteis quando vamos executar um gridsearch, por exemplo. Com essa classe podemos acessar os valores que estão definidos dentro do método __init__, bem como atribuir novos valores apra ele. 

Já TransformerMixin tras implementado o método fit_transform(), que nada mais é do que aplicar o método fit() e, logo em seguida, o método transform.

Esse link pode ajudar a esclarecer melhor as coisas, caso ainda tenham dúvidas: [link](https://github.com/ageron/handson-ml/issues/391)

In [ ]:
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import OrdinalEncoder #skl 0.20>
num_attribs = list(data_num)
cat_attribs = ['month','day']
num_pipeline = Pipeline([
    ('selector', DataFrameSelector(num_attribs)),
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')),
])
cat_pipeline = Pipeline([
    ('selector', DataFrameSelector(cat_attribs)),
    ('categorial_encoder', OrdinalEncoder())
    ])
full_pipeline = FeatureUnion(transformer_list=[
    ("num_pipeline", num_pipeline),
    ("cat_pipeline", cat_pipeline),
])

In [ ]:
data_prepared = full_pipeline.fit_transform(data) #retorna um numpy.array

In [ ]:
# Agora temos nossos dados preparados.
data_prepared[0:5,:]